# SOAP Note Generation

This notebook performs step 1 of the Brief Hospital Course pipeline, in which we generate a SOAP note for each service based on a list of information that has been passed into a GPT-3.5 model. 

In [11]:
import pandas as pd
import ast
import numpy as np

from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

import os
import openai

In [12]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [21]:
N = 5

## Read in Radiology Reports

In [19]:
radiology = pd.read_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/radiology.csv.gz")

## Read in Structured Data

In [5]:
# discharge summaries
discharges = pd.read_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/discharge.csv.gz")

# ed stays
edstays = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/edstays.csv.gz')

# triage
triage = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/triage.csv.gz')

# ward transfers
transfers = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/transfers.csv.gz')

# higher-level services (ICU, CARD, etc)
services = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/services.csv.gz')

# get patient info
pts = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/patients.csv.gz')

# admission demographics
admissions = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/admissions.csv.gz')

# procedures
procs = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/procedures_icd.csv.gz')
procs_icd = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_icd_procedures.csv.gz')

# diagnoses
diags = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/diagnoses_icd.csv.gz')
diags_icd = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_icd_diagnoses.csv.gz')

# meds
med_orders = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/emar.csv.gz')


In [70]:

# # procedures
# procs = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/procedures_icd.csv.gz')
# procs_icd = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_icd_procedures.csv.gz')

# # diagnoses
# diags = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/diagnoses_icd.csv.gz')
# diags_icd = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_icd_diagnoses.csv.gz')


In [13]:
prescriptions = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/prescriptions.csv.gz')
labs = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/labevents.csv.gz')
microbio = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/microbiologyevents.csv.gz')

/tmp/ipykernel_2337815/295397496.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/prescriptions.csv.gz')
/tmp/ipykernel_2337815/295397496.py:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  microbio = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/microbiologyevents.csv.gz')


In [37]:
labs_icd = pd.read_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_labitems.csv.gz")

In [50]:
labs = labs.merge(labs_icd, on="itemid", how="left")

### Clean up/type cast data

In [14]:
med_orders = med_orders[~med_orders['charttime'].isna() & 
                        ~med_orders['medication'].isna() & 
                        ~med_orders['event_txt'].isna()]

In [15]:
procs = procs.astype({"chartdate":"datetime64[ns]"})
med_orders = med_orders.astype({"charttime":"datetime64[ns]",
                  "scheduletime":"datetime64[ns]",
                  "storetime":"datetime64[ns]",})

discharges = discharges.astype({"charttime":"datetime64[ns]",
                               "storetime":"datetime64[ns]"})

In [29]:
prescriptions = prescriptions.astype({"starttime":"datetime64[ns]",
                      "stoptime":"datetime64[ns]"})
labs = labs.astype({"charttime":"datetime64[ns]",
                      "storetime":"datetime64[ns]"})

microbio = microbio.astype({"chartdate":"datetime64[ns]",
                      "charttime":"datetime64[ns]",
                           "storetime":"datetime64[ns]",
                      "charttime":"datetime64[ns]"})

In [71]:
# drop any potential repeats
procs_icd = procs_icd.groupby(["icd_code", "icd_version"]).first().reset_index()
diags_icd = diags_icd.groupby(["icd_code", "icd_version"]).first().reset_index()

In [72]:
# grab long_titles for procs/diags
procs = procs.merge(procs_icd, on=["icd_code", "icd_version"], how="left")
diags = diags.merge(diags_icd, on=["icd_code", "icd_version"], how="left")


### Define Extraction Functions

In [30]:
# generate free text sections of tables concatenating columns
med_orders['admin_text'] = med_orders['medication'] + " at " + med_orders['charttime'].dt.strftime('%B %d, %Y, %r')
# TODO: Add the right information in here that we actually want for the meds
prescriptions['text'] = prescriptions['drug'] + " " + prescriptions['prod_strength']


In [35]:
with pd.option_context("display.max_columns", None):
    display(prescriptions.sample(2))

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,text
15265619,19906572,20909471,94135471,19906572-1000,1000.0,P99E90,2135-11-06 05:00:00,2135-11-08 18:00:00,BASE,SW,KCLBASE,NaN,0.000000e+00,100ml Bag,NaN,200,mL,200,mL,NaN,IV,SW 100ml Bag
3784434,12462463,24666064,48450074,12462463-134,134.0,P12FN3,2117-07-16 08:00:00,2117-07-16 08:00:00,MAIN,Potassium Chloride,KCL20P,001262,6.954304e+10,20mEq Packet,NaN,40,mEq,2,PKT,1.0,PO,Potassium Chloride 20mEq Packet


In [52]:
# TODO: figure out how to better encapsulate lab values
labs['text'] = labs['category'] + " " + labs['fluid'] + " " + labs['label'] + ": " + labs['value'] + " " + labs['valueuom']

In [59]:
# TODO: check for those patients that don't have comments
microbio['text'] = microbio['test_name'] + ": " + microbio['comments']

In [56]:
microbio.isna().sum()

microevent_id                0
subject_id                   0
hadm_id                1830396
micro_specimen_id            0
order_provider_id      2272201
chartdate                    0
charttime               257699
spec_itemid                  0
spec_type_desc               1
test_seq                     0
storedate                12251
storetime                25186
test_itemid                  0
test_name                    0
org_itemid             1942044
org_name               1942044
isolate_num            1942044
quantity               3228683
ab_itemid              2121435
ab_name                2121435
dilution_text          2148780
dilution_comparison    2149420
dilution_value         2149420
interpretation         2121435
comments                990931
dtype: int64

In [76]:
# create initial input text prompt
def get_demos(subject_id):
    # has gender, anchor-age, date-of-death if exists
    return pts[pts['subject_id'] == subject_id].squeeze()
    
def get_transfers(hadm_id):
    return transfers[transfers['hadm_id'] == hadm_id].sort_values("intime").squeeze()

def get_triage_info(stay_id):
    return triage[triage['stay_id'] == stay_id]
    
def get_procs(hadm_id):
    adm_procs = procs[procs['hadm_id'] == hadm_id]
    return adm_procs.sort_values("seq_num")

def get_diags(hadm_id):
    adm_diags = diags[diags['hadm_id'] == hadm_id]
    return adm_diags.sort_values("seq_num")

def get_med_orders(hadm_id):
    med_admin = med_orders[(med_orders['hadm_id'] == hadm_id) & (med_orders['event_txt'] == "Administered")]
    med_admin['admin_text'] = med_admin['medication'] + " at " + med_admin['charttime'].dt.strftime('%B %d, %Y, %r')
    return med_admin

def get_prescriptions(hadm_id):
    adm_prescriptions = prescriptions[(prescriptions['hadm_id'] == hadm_id)]
    
def get_labs(hadm_id):
    adm_labs = labs[(labs['hadm_id'] == hadm_id)]

def get_microbio(hadm_id):        
    adm_microbio = microbio[(microbio['hadm_id'] == hadm_id)]


def get_med_orders_within_service(hadm_id, transfer_event):
    med_admin = med_orders[(med_orders['hadm_id'] == hadm_id) & (med_orders['event_txt'] == "Administered")]
    med_admin = med_admin.sort_values("emar_seq")
    
    adm_diags_in_unit = med_admin[(med_admin['charttime'] > transfer_event['intime'])
                                & (med_admin['charttime'] < transfer_event['outtime'])]

    # adm_diags_in_unit['admin_text'] = adm_diags_in_unit['medication'] + " at " + adm_diags_in_unit['charttime'].dt.strftime('%B %d, %Y, %r')
    
    return adm_diags_in_unit

def get_procs_within_service(hadm_id, transfer_event):
    adm_procs = procs[procs['hadm_id'] == hadm_id]
    adm_procs = adm_procs.sort_values("seq_num")
    
    adm_procs_in_unit = adm_procs[(adm_procs['chartdate'] > transfer_event['intime'])
                                & (adm_procs['chartdate'] < transfer_event['outtime'])]
    return adm_procs_in_unit


def get_prescriptions_within_service(hadm_id, transfer_event):
    adm_prescriptions = prescriptions[(prescriptions['hadm_id'] == hadm_id)]
    # adm_prescriptions['text'] = adm_prescriptions['drug'] + " " + adm_prescriptions['prod_strength']
    adm_prescriptions_in_unit = adm_prescriptions[(adm_prescriptions['starttime'] > transfer_event['intime'])
                                & (adm_prescriptions['starttime'] < transfer_event['outtime'])]
    return adm_prescriptions_in_unit
    
def get_labs_within_service(hadm_id, transfer_event):
    adm_labs = labs[(labs['hadm_id'] == hadm_id)]
    adm_labs = adm_labs[(adm_labs['charttime'] > transfer_event['intime'])
                                & (adm_labs['charttime'] < transfer_event['outtime'])]
    return adm_labs
    
def get_microbio_within_service(hadm_id, transfer_event):
    adm_microbio = microbio[(microbio['hadm_id'] == hadm_id)]
    # TODO: we need to do something because chartdate is required, but charttime isn't, so what do we do when we don't have times? 
    adm_microbio = adm_microbio[(adm_microbio['chartdate'] > transfer_event['intime'])
                                & (adm_microbio['chartdate'] < transfer_event['outtime'])]
    return adm_microbio
    

In [77]:
def create_pt_prompt_per_service(discharge_row):

    demos = get_demos(discharge_row['subject_id'])
    if demos.empty:
        age = r"[UNKNOWN AGE]"
        sex = r"[UNKNOWN SEX]"
    else:
        age = demos['anchor_age']
        sex = demos['gender']

    pt_edstays = edstays[edstays['hadm_id'] == discharge_row['hadm_id']]

    ccs = []
    for stay_id in pt_edstays['stay_id'].tolist():
        triage_info = get_triage_info(stay_id)
        ccs.append(triage_info['chiefcomplaint'].squeeze())
        
    chief_complaints = ", ".join(ccs)

    if sex:
        pronoun = ["he","his"] if sex == "M" else ['she',"her"]
    else:
        pronoun = ["they", "their"]

    # transfers with dates
    transfers = get_transfers(discharge_row['hadm_id'])

    # get stay admission diagnoses
    diags = get_diags(discharge_row['hadm_id'])

    init_prompt = f"___ is a {age} year old {sex} that initially presented to the ED with {chief_complaints}. By the end of {pronoun[1]} hospital stay, {pronoun[0]} was given the following diagnoses: {', '.join(diags['long_title'])} in order of importance to this admission. "
    transfer_service_prompts = []

    for index, row in transfers.iterrows():
        if row['eventtype'] == "discharge":
             transfer_service_prompts.append("")
        else:
            # TODO: add from here. 
            within_service_prompt = f"The patient was transferred and stayed in the {row['careunit']} ward between {row['intime']} and {row['outtime']}. ___ received "
            
            procs = get_procs_within_service(discharge_row['hadm_id'], row)
            # med_orders = get_med_orders_within_service(discharge_row['hadm_id'], row)
            prescriptions = get_prescriptions_within_service(discharge_row['hadm_id'], row)
            labs = get_labs_within_service(discharge_row['hadm_id'], row)
            microbio = get_microbio_within_service(discharge_row['hadm_id'], row)
            
            within_service_prompt = within_service_prompt + f"the following procedures (ordered by priority): {', '.join(procs['long_title'].tolist())}. "
            # within_service_prompt = within_service_prompt + f"{pronoun[0]} also received the following medications (ordered chronologically) during the service: {', '.join(med_orders['medication'].tolist())}. "            
            within_service_prompt = within_service_prompt + f"\n------------------------\n{pronoun[0]} also received the following medications (ordered chronologically) during the service: {', '.join(prescriptions['text'].tolist())}. "
            within_service_prompt = within_service_prompt + f"\n------------------------\nThe following labs were also drawn during the service: {', '.join(labs['text'].tolist())}. "
            within_service_prompt = within_service_prompt + f"\n------------------------\nThe physician also ordered the following microbiology cultures during the service: {', '.join(microbio['text'].tolist())}. "
            
            full_prompt = init_prompt + within_service_prompt + f"Given this information, please generate a progress note for this patient for their care during this part of their hospital course staying in the {row['careunit']} ward."
            transfer_service_prompts.append(full_prompt)

    transfers['service_prompts'] = transfer_service_prompts
    
    return transfers
    


## Generate Input Data List Prompt

In [83]:
# TODO: why are there so many NAs?: 22680492
labs = labs[~labs['text'].isna()]

In [84]:
service_prompts = create_pt_prompt_per_service(discharges.sample(1).squeeze())

In [87]:
print(service_prompts['service_prompts'].iloc[1])

___ is a 83 year old F that initially presented to the ED with Abd pain, Vomiting. By the end of her hospital stay, she was given the following diagnoses: Sepsis, unspecified organism, Acidosis, Other ascites, Chronic atrial fibrillation, Chronic obstructive pulmonary disease, unspecified, Urinary tract infection, site not specified, Severe sepsis without septic shock, Unspecified Escherichia coli [E. coli] as the cause of diseases classified elsewhere, Do not resuscitate, Long term (current) use of anticoagulants, Personal history of nicotine dependence, Unspecified dementia without behavioral disturbance, Major depressive disorder, single episode, unspecified in order of importance to this admission. The patient was transferred and stayed in the Medicine ward between 2174-10-14 23:08:00 and 2174-10-21 12:26:37. ___ received the following procedures (ordered by priority): Insertion of Infusion Device into Superior Vena Cava, Percutaneous Approach. 
------------------------
she also re

## Create SOAP notes from GPT API

In [20]:
import openai
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
engine = "decile-gpt-35-turbo-16k"


In [91]:
# drop all eventtypes of type: discharge, since we don't need a note for that
service_prompts = service_prompts[service_prompts['eventtype'] != "discharge"]

In [96]:
gpt_inputs = []
for _, row in service_prompts.iterrows():
    message_text = [{"role":"system","content":f"You are a physician that is reviewing a patient's medical record during their stay in your ward: ({row['careunit']}) and writing a SOAP Note based on this information."},]
    
    gpt_service_prompt = {"role":"user",
                     "content":row['service_prompts']}
    
    message_text.append(gpt_service_prompt)
    
    # print(f"Deprescribe Prompt: {message_text}")
    gpt_inputs.append(message_text)

In [97]:
completions = []
for input in gpt_inputs:
    completion = openai.ChatCompletion.create(
      engine=engine,
      messages = input,
    )
    completions.append(completion['choices'][0]['message']['content'])

In [98]:
service_prompts['gpt_SOAP_note'] = completions

/tmp/ipykernel_2337815/611083610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  service_prompts['gpt_SOAP_note'] = completions


In [100]:
service_prompts.to_csv("soap_note_sample.csv")